<a href="https://colab.research.google.com/github/hishaam19/RockID/blob/TrainingModelsAlex/Usman_Training_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import keras
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
import os
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [12]:
img_width=80; img_height=80
batch_size=32
NumOfclasses=134

TRAIN_DIR = "/content/drive/MyDrive/Datasets/train"
VAL_DIR = "/content/drive/MyDrive/Datasets/val"
TEST_DIR= "/content/drive/MyDrive/Datasets/test"

In [13]:
#preprocessing datasets
train_data = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,

                                    vertical_flip=False  )

val_data = ImageDataGenerator(rescale=1./255)
test_data = ImageDataGenerator(rescale=1./255)

train_gen = train_data.flow_from_directory(TRAIN_DIR,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True,
                                                target_size=(img_height, img_width))





val_gen = val_data.flow_from_directory(VAL_DIR,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True,
                                                target_size=(img_height, img_width)
                                                            )
test_gen = test_data.flow_from_directory(TEST_DIR,
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True,
                                                target_size=(img_height, img_width)
                                                            )

Found 3082 images belonging to 134 classes.
Found 847 images belonging to 134 classes.
Found 556 images belonging to 134 classes.


In [14]:
#create training model

model=Sequential()
pretrained_model=tf.keras.applications.vgg16.VGG16(include_top=False,
              weights='imagenet',
              input_shape=(img_height,img_width,3))
for layer in pretrained_model.layers:
  layer.trainable = False

model.add(pretrained_model)
model.add(Flatten()) #Research DO NOT CHANGE
model.add(Dense(1024, activation='relu'))  #Research
model.add(Dense(512, activation='relu'))   #Reasearch
model.add(Dense(19, activation='softmax'))
model.summary()
model.compile(optimizer=Adam(learning_rate=0.001),
      loss='categorical_crossentropy',
      metrics =['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 134)               68742     
                                                                 
Total params: 17,406,406
Trainable params: 2,691,718
Non-trainable params: 14,714,688
___________________________________

In [ ]:
history = model.fit(train_gen,
                                epochs=30,
                                verbose=1,
                                validation_data=val_gen,

                                )

Epoch 1/30
64/97 [==================>...........] - ETA: 7:37 - loss: 4.5783 - accuracy: 0.0542

In [ ]:
print("Evaluate test data")
#test model
model.evaluate(test_gen)
model.save("test.h5")
#Plot accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()